# RAG Evaluation Using RAGAS

## Introduction to Evaluation
Evaluation in the context of Retrieval-Augmented Generation (RAG) systems involves assessing the performance of both the retrieval component (how well relevant documents are fetched from a knowledge base)
and the generation component (how accurate, relevant, and coherent the generated responses are). A RAG system combines a retriever (e.g., a vector store like FAISS) with a language model (e.g., AzureChatOpenAI) to provide contextually informed responses, reducing issues like hallucinations (incorrect or fabricated information).

## Why Do We Use Evaluation?
- Evaluation is critical for the following reasons:
- Quality Assurance: Ensures the RAG system delivers accurate, relevant, and trustworthy responses.
- System Improvement: Identifies weaknesses in retrieval (e.g., irrelevant documents) or generation (e.g., unfaithful answers), guiding optimizations like better embeddings or prompt engineering.
- Performance Monitoring: Quantifies system performance to track improvements or regressions over time.
- Stakeholder Confidence: Provides metrics to demonstrate the system's reliability to stakeholders or end-users.

### The RAGAS framework (Retrieval Augmented Generation Assessment) is used to evaluate RAG systems. It provides metrics like:
- Faithfulness: Measures if the generated answer is factually grounded in the retrieved context.
- Answer Relevancy: Assesses if the answer directly addresses the user's query.
- Context Precision: Checks if the retrieved context contains relevant information with minimal noise.
- Context Recall: Ensures all necessary information is retrieved (requires ground truth).

This notebook sets up a RAG system using AzureChatOpenAI, AzureOpenAIEmbeddings, and FAISS, generates a synthetic test dataset, and evaluates the system using RAGAS.

Loads environment variables (e.g., API keys) from a .env file for secure configuration.

In [1]:
!pip install pymupdf --quiet

In [2]:
import os
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os 

load_dotenv()

MODEL_NAME = "gpt-4.1-mini"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"


Initializes AzureChatOpenAI for response generation and AzureOpenAIEmbeddings for creating document embeddings.

In [3]:
llm = AzureChatOpenAI(azure_deployment=MODEL_NAME)

embeddings = AzureOpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

dir_path = r"datasets/supply_chain"
index_path = r"RAG/index/"
collection_name = "supply_chain"

## Document Loading
This section loads PDF documents from a directory

In [4]:
def load_documents():
    """
    Load PDF documents from the specified directory using PyMuPDFLoader.
    
    Returns:
        list: A list of loaded documents.
    
    Raises:
        FileNotFoundError: If the directory does not exist.
        Exception: For other loading errors.
    """
    if not os.path.exists(dir_path):
        raise FileNotFoundError(f"Directory not found: {dir_path}")
    try:
        loader = DirectoryLoader(dir_path, loader_cls=PyMuPDFLoader)
        return loader.load()
    except Exception as e:
        raise e

def split_documents(documents):
    """
    Split documents into smaller chunks using RecursiveCharacterTextSplitter.
    
    Args:
        documents (list): List of documents to split.
    
    Returns:
        list: A list of document chunks. Returns empty list if no documents.
    """
    try:
        if not documents:
            return []
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        return text_splitter.split_documents(documents)
    except Exception as e:
        print(f"Error splitting documents: {str(e)}")
        return []

# Load and split documents
documents = load_documents()
documents = split_documents(documents)
print(f"Loaded and split {len(documents)} document chunks.")

Loaded and split 900 document chunks.


## Vector Store Creation

Now splits documents into chunks, and creates a FAISS vector store for retrieval.

In [9]:
def create_vectorstore(documents):
    """
    Create and save a new FAISS vector store from documents.
    
    Args:
        documents (list): List of document objects to convert to vectors.
    
    Returns:
        None: If successful, else Exception.
    """
    try:
        os.makedirs(index_path, exist_ok=True)
        vectorstore = Chroma.from_documents(documents, embeddings,persist_directory=index_path,
                                             collection_name=collection_name)
        save_vectorstore(vectorstore)
        print("Vector store created successfully.")
    except Exception as e:
        print(e)
        return e

def save_vectorstore(vectorstore):
    """
    Save the FAISS vector store to the specified path.
    
    Args:
        vectorstore (FAISS): The vector store to save.
    
    Returns:
        None: If successful, else Exception.
    """
    try:
        vectorstore.save_local(index_path)
    except Exception as e:
        return e

def load_vectorstore():
    """
    Load an existing FAISS vector store.
    
    Returns:
        FAISS: Loaded vector store, else Exception.
    """
    try:
        return Chroma(embedding_function=embeddings,persist_directory=index_path,
                      collection_name=collection_name)
    except Exception as e:
        return e

# Load or create vector store
if os.path.exists(index_path) and any(os.listdir(index_path)):
    vectorstore = load_vectorstore()
    vectorstore_retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
    print("Vector store loaded successfully.")
else:
    create_vectorstore(documents)
    vectorstore = load_vectorstore()
    vectorstore_retriever = vectorstore.as_retriever(search_kwargs={'k': 5})
    print("Created and loaded new vector store.")

Vector store loaded successfully.


### Explanation:

- Document Loading: Uses DirectoryLoader with PyMuPDFLoader to load PDFs from the data directory.
- Document Splitting: Splits documents into chunks (500 characters, 200 overlap) for efficient retrieval.
- Vector Store: Creates a FAISS index from document embeddings or loads an existing one from the index directory.
- Retriever: Configures the vector store as a retriever, fetching the top 5 relevant documents for a query.

## RAG Chain Setup
This section defines a RAG chain that validates queries, retrieves relevant documents, and generates answers using the AzureChatOpenAI model.

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def validate_query(query):
    """
    Validates a user's query by ensuring it is not empty and has at least 15 characters.
    
    Args:
        query (str): The input query.
    
    Returns:
        str: The query if valid, or an error message if invalid.
    """
    try:
        if not query:
            return "Query cannot be empty, enter a valid query."
        elif len(query) < 15:
            return "Query is too short, enter a valid query."
        else:
            return query
    except Exception as e:
        return str(e)

def create_rag_chain(query, relevant_documents):
    """
    Creates and executes a RAG chain to answer a query using retrieved documents.
    
    Args:
        query (str): The user query.
        relevant_documents (list): List of retrieved document chunks.
    
    Returns:
        str: The generated response or an error message.
    """
    try:
        prompt_template = """
        Only based on the provided documents, answer the question in points. Do not mention from which document the answer is derived.
        Question: {query}
        Documents: {docs}
        Note: You are a supply chain assistant. If the query is not related to supply chain or the documents do not provide the necessary information, return "Invalid Query".
        """
        prompt = ChatPromptTemplate.from_template(prompt_template)
        valid_query = validate_query(query)
        rag_chain = prompt | llm | StrOutputParser()
        return rag_chain.invoke({"query": valid_query, "docs": relevant_documents})
    except Exception as e:
        return str(e)

# Test the RAG chain
query = "What is Supply Chain?"
relevant_documents = vectorstore_retriever.invoke(query)
response = create_rag_chain(query, relevant_documents)
print("RAG Chain Response:")
print(response)

RAG Chain Response:
- A supply chain is a network of partners who collectively convert a basic commodity (upstream) into a finished product (downstream) that is valued by end-customers.
- It involves managing material flows from raw materials (like a bauxite mine) through various stages of conversion to finished goods (like a can of cola).
- The supply chain also includes the management of returns at each stage, such as rejected materials or recycling of finished products.
- It can be described as a sequence of linked organizations (a chain) or a more complex network where organizations are cross-linked with two-way exchanges.
- It encompasses the management processes of planning, sourcing, making, delivering, and returning goods.


### Explanation:

- Query Validation: Ensures the query is non-empty and at least 15 characters long.
- RAG Chain: Constructs a prompt that instructs the model to answer in bullet points, using only the retrieved documents, and to return "Invalid Query" if the query is unrelated to supply chain or unsupported by the documents.
- Execution: Combines the prompt, AzureChatOpenAI model, and string output parser to generate a response.
- Test: Runs a sample query to verify the RAG chain's functionality.

## Generating Synthetic Test Data with RAGAS
To evaluate the RAG system, we need a test dataset with questions, answers, contexts, and ground truth. RAGAS's TestsetGenerator can create synthetic data from documents.

In [14]:
!pip install ragas rapidfuzz --quiet

In [15]:
import random
from ragas.testset import TestsetGenerator

# Configure the test set generator
testset_generator = TestsetGenerator.from_langchain(
    llm=llm,
    embedding_model=embeddings
)

# Randomly sample a subset of documents (e.g., 50 out of 902 chunks)
sample_size = 10  # Adjust based on your needs
random.seed(42)  # For reproducibility
sampled_documents = random.sample(documents, min(sample_size, len(documents)))

# Generate test dataset with reduced test_size
testset = testset_generator.generate_with_langchain_docs(sampled_documents, 5)

# Convert test dataset to evaluation format
eval_data = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": []
}

for testcase in testset.samples:
    relevant_docs = vectorstore_retriever.invoke(testcase.eval_sample.user_input)
    answer = create_rag_chain(testcase.eval_sample.user_input, relevant_docs)
    eval_data["question"].append(testcase.eval_sample.user_input)
    eval_data["answer"].append(answer)
    eval_data["contexts"].append([doc.page_content for doc in relevant_docs])
    eval_data["ground_truth"].append(testcase.eval_sample.reference)

print(f"Generated {len(eval_data['question'])} test cases.")

Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]       Node 4a32b931-dd50-4ffe-87d9-0fd2d864664a does not have a summary. Skipping filtering.
Node f2bf59b0-b22b-4e45-897a-2bad546dcef1 does not have a summary. Skipping filtering.
Node b8d0952a-dc30-4b38-9c56-ede53db5a860 does not have a summary. Skipping filtering.
Node 047ff819-9b7e-45ab-bdae-ae21815123f3 does not have a summary. Skipping filtering.
Node 6ea3f6ac-263f-4978-8ea4-9d9248f0f765 does not have a summary. Skipping filtering.
Node 34d5122b-53c8-4d1a-b463-fa1d2b7ef9ed does not have a summary. Skipping filtering.
Generating Samples: 100%|██████████| 6/6 [00:54<00:00,  9.06s/it]


Generated 6 test cases.


### Explanation:

- TestsetGenerator: Uses AzureChatOpenAI for generating and critiquing test cases, with AzureOpenAIEmbeddings for document embeddings.
- Test Data Generation: Creates test cases with a mix of random samples
- Evaluation Dataset: For each test case, retrieves relevant documents, generates an answer using the RAG chain, and collects the question, answer, contexts, and ground truth.
- Output: Stores the data in a dictionary format suitable for RAGAS evaluation.

## RAG Evaluation with RAGAS
This section evaluates the RAG system using RAGAS metrics: faithfulness, answer relevancy, context precision, and context recall.

In [16]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from datasets import Dataset
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Convert evaluation data to Hugging Face Dataset
eval_dataset = Dataset.from_dict(eval_data)

# Wrap AzureChatOpenAI for RAGAS compatibility
evaluator_llm = LangchainLLMWrapper(llm)
evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings)

# Run evaluation
results = evaluate(
    dataset=eval_dataset,
    metrics=[
        faithfulness,       # Checks if the answer is grounded in the context
        answer_relevancy,   # Checks if the answer addresses the question
        context_precision,  # Checks if retrieved context is relevant
        context_recall      # Checks if all necessary information is retrieved
    ],
    llm=evaluator_llm,
    embeddings=evaluator_embeddings,
    show_progress=True
)

# Print evaluation results
print("RAGAS Evaluation Results:")
print(results)

Evaluating:  71%|███████   | 17/24 [02:18<01:51, 15.95s/it]Exception raised in Job[2]: TimeoutError()
Exception raised in Job[6]: TimeoutError()
Exception raised in Job[10]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Evaluating: 100%|██████████| 24/24 [03:05<00:00,  7.74s/it]


RAGAS Evaluation Results:
{'faithfulness': 0.8163, 'answer_relevancy': 0.9332, 'context_precision': 1.0000, 'context_recall': 0.9583}


### Explanation:

- Dataset Conversion: Converts the evaluation data into a Hugging Face Dataset for RAGAS.
- LLM Wrapper: Wraps AzureChatOpenAI with LangchainLLMWrapper for compatibility with RAGAS.
- Metrics: Evaluates the RAG system on:
    - Faithfulness: Ensures answers are factually consistent with the context.
    - Answer Relevancy: Measures how well answers address the query.
    - Context Precision: Assesses the relevance of retrieved documents.
    - Context Recall: Checks if all necessary information is retrieved (uses ground truth).
- Results: Outputs scores (0 to 1) for each metric, where higher is better.